In [9]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

def compute_path_hist(file):
    paths = np.reshape(np.fromfile(file, dtype=np.float32), (3,-1,8))
    paths = paths.transpose(1,2,0)
    paths = paths[:50]

    filtered_paths = []
    for path in paths:
        valid = np.argmin(path[:,2] < -15)
        if valid == 0:
            valid = len(path) - 1
        
        filtered_paths.append(path)

    filtered_paths = np.array(filtered_paths)

    print(filtered_paths.shape)

    return np.histogram2d(
        filtered_paths[:,2,0], 
        filtered_paths[:,2,1], bins=(50, 50), density=True, range=((-16,-5), (0,15)))

def plot_paths_hist_diff(file0,file1,ax):
    histA, _, _ = compute_path_hist(file0)
    histB, X, Y = compute_path_hist(file1)
    pc = ax.pcolormesh(X, Y, (histA.T-histB.T)**2)
    plt.colorbar(pc,ax=ax)
    ax.set_aspect('equal')

def plot_paths_hist(file, ax):
    hist, X, Y = compute_path_hist(file)
    pc = ax.pcolormesh(X, Y, hist.T, vmin=0, vmax=0.2)
    ax.set_aspect('equal')

def plot_paths(file, ax, color, xoffset = 0):
    paths = np.reshape(np.fromfile(file, dtype=np.float32), (3,-1,8))
    paths = paths.transpose(1,2,0)
    paths = paths[:500]

    for path in paths:
        valid = np.argmin(np.any(path != 0, axis=-1))
        if valid == 0:
            valid = len(path) - 1
        
        first = np.minimum(valid, 0)
        valid = np.minimum(valid, 3)
        path = path[first:valid]
        ax.scatter(-path[:,0]+xoffset, path[:,2], path[:,1], alpha=0.05, color=color)
        ax.plot(-path[:,0]+xoffset, path[:,2], zs=path[:,1], alpha=0.05, color=color)

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    ax.set_xlim(-20, 20)
    ax.set_ylim(-35, 10)
    ax.set_zlim(-2, 10)
    ax.grid(True, axis="x")
    #ax.set_aspect('equal')

fig = plt.figure(figsize=(20,10))

axs = [fig.add_subplot(131, projection="3d"),
       fig.add_subplot(132),
       fig.add_subplot(133)]

plot_paths("testing/intersections/rough-se-aniso-pseudo/medium-paths001.bin", axs[0], color="blue", xoffset = -0.5)
plot_paths("testing/intersections/rough-se-iso-pseudo/medium-paths.bin", axs[0], color="red", xoffset = 0)
plot_paths("testing/intersections/rough-se/medium-paths001.bin", axs[0], color="green", xoffset = 0.5)
#plot_paths("testing/intersections/gp-medium-sphere-softlight-rough-elephant/medium-paths002.bin", axs[0], color="green", xoffset = -0.3)
#plot_paths("testing/intersections/surface-paths003.bin", axs[0], color="red", xoffset = 5)

#plot_paths_hist("testing/intersections/gp-medium-sphere-softlight-rough/medium-paths003.bin", axs[1])
#plot_paths_hist("testing/intersections/gp-medium-sphere-softlight-rough-elephant/medium-paths001.bin", axs[2])
#plot_paths_hist_diff("testing/intersections/gp-medium-sphere-softlight-rough-ref/surface-paths002.bin",
#                     "testing/intersections/gp-medium-sphere-softlight-rough/medium-paths003.bin", 
#                     axs[2])

#lot_paths_hist_diff("testing/intersections/gp-medium-sphere-softlight-rough-ref/surface-paths002.bin",
#                     "testing/intersections/gp-medium-sphere-softlight-rough-elephant/medium-paths002.bin", 
#                     axs[1])

fig.tight_layout()


plt.show()
